In [1]:
import sys
# set python syspath to point out location of our self-writing module
sys.path.append("/home/ponshane/work_dir/CLTM/src/codebase/")
from helper import *

import configparser
import pymongo
from pymongo import MongoClient
from gensim.models import KeyedVectors
import pandas as pd
import os

In [2]:
### init and read config
config = configparser.ConfigParser()
config.read('../config.ini')

MongoDB = config["MLDoc"]["Database"]
MongoUser = config["MLDoc"]["User"]
MongoPW = config["MLDoc"]["PW"]

###連接MONGO
uri = "mongodb://" + MongoUser + ":" + MongoPW + "@140.117.69.70:30241/" +\
MongoDB + "?authMechanism=SCRAM-SHA-1"

client = MongoClient(uri)
db = client.MLDoc

# tag train/dev/test on Chinese Corpus
**Already labeled**

In [13]:
# fo = open("/home/ponshane/work_dir/MLDoc/mldoc-indices/chinese.train.10000", "r")
 
# for line in fo.readlines():
    
#     line = line.split("-")
#     # 確認是否為 folder-docID 格式
#     assert len(line) == 2

#     status = db.Chinese.update_one({"$and":[{"Sub_corpus":line[0]},{"File_name":line[1].strip("\n")}]},
#                        {"$set": {"Data_type":"train"}})
    
# fo.close()

In [31]:
# fo = open("/home/ponshane/work_dir/MLDoc/mldoc-indices/chinese.dev", "r")
 
# for line in fo.readlines():
    
#     line = line.split("-")
#     # 確認是否為 folder-docID 格式
#     assert len(line) == 2

#     status = db.Chinese.update_one({"$and":[{"Sub_corpus":line[0]},{"File_name":line[1].strip("\n")}]},
#                        {"$set": {"Data_type":"dev"}})
    
# fo.close()

In [32]:
# fo = open("/home/ponshane/work_dir/MLDoc/mldoc-indices/chinese.test", "r")
 
# for line in fo.readlines():
    
#     line = line.split("-")
#     # 確認是否為 folder-docID 格式
#     assert len(line) == 2

#     status = db.Chinese.update_one({"$and":[{"Sub_corpus":line[0]},{"File_name":line[1].strip("\n")}]},
#                        {"$set": {"Data_type":"test"}})
    
# fo.close()

# tag train/dev/test on English Corpus
**Already labeled**

In [14]:
# fo = open("/home/ponshane/work_dir/MLDoc/mldoc-indices/english.train.10000", "r")
 
# for line in fo.readlines():
    
#     line = line.split("-")
#     # 確認是否為 folder-docID 格式
#     assert len(line) == 2

#     status = db.English.update_one({"$and":[{"Sub_corpus":line[0]},{"File_name":line[1].strip("\n")}]},
#                        {"$set": {"Data_type":"train"}})
    
# fo.close()

In [38]:
# fo = open("/home/ponshane/work_dir/MLDoc/mldoc-indices/english.dev", "r")
 
# for line in fo.readlines():
    
#     line = line.split("-")
#     # 確認是否為 folder-docID 格式
#     assert len(line) == 2

#     status = db.English.update_one({"$and":[{"Sub_corpus":line[0]},{"File_name":line[1].strip("\n")}]},
#                        {"$set": {"Data_type":"dev"}})
    
# fo.close()

In [39]:
# fo = open("/home/ponshane/work_dir/MLDoc/mldoc-indices/english.test", "r")
 
# for line in fo.readlines():
    
#     line = line.split("-")
#     # 確認是否為 folder-docID 格式
#     assert len(line) == 2

#     status = db.English.update_one({"$and":[{"Sub_corpus":line[0]},{"File_name":line[1].strip("\n")}]},
#                        {"$set": {"Data_type":"test"}})
    
# fo.close()

# tag train/dev/test on Japanese Corpus
**Already labeled**

In [15]:
# fo = open("/home/ponshane/work_dir/MLDoc/mldoc-indices/japanese.train.10000", "r")
 
# for line in fo.readlines():
    
#     line = line.split("-")
#     # 確認是否為 folder-docID 格式
#     assert len(line) == 2

#     status = db.Japanese.update_one({"$and":[{"Sub_corpus":line[0]},{"File_name":line[1].strip("\n")}]},
#                        {"$set": {"Data_type":"train"}})
    
# fo.close()

In [41]:
# fo = open("/home/ponshane/work_dir/MLDoc/mldoc-indices/japanese.dev", "r")
 
# for line in fo.readlines():
    
#     line = line.split("-")
#     # 確認是否為 folder-docID 格式
#     assert len(line) == 2

#     status = db.Japanese.update_one({"$and":[{"Sub_corpus":line[0]},{"File_name":line[1].strip("\n")}]},
#                        {"$set": {"Data_type":"dev"}})
    
# fo.close()

In [42]:
# fo = open("/home/ponshane/work_dir/MLDoc/mldoc-indices/japanese.test", "r")
 
# for line in fo.readlines():
    
#     line = line.split("-")
#     # 確認是否為 folder-docID 格式
#     assert len(line) == 2

#     status = db.Japanese.update_one({"$and":[{"Sub_corpus":line[0]},{"File_name":line[1].strip("\n")}]},
#                        {"$set": {"Data_type":"test"}})
    
# fo.close()

# Select data where Data_type = train or test or dev
* only 6000*2 docs in total(1000 train, 1000 dev, 4000 test in each language)
* **English corpus 需要以project方式撈取，否則 word2vec model 有 stop words 此處需要解查** 

# Chinese-English Corpus

In [3]:
#format => [{_id, extracted_text, Class, Data_type, language}, ... ]
corpus = []

# load back model to filter tokens
Concatenated_model = KeyedVectors.load_word2vec_format("/home/ponshane/work_dir/CLTM-Experiments/Data/EN-ZH-MLDoc/word-vectors/Chinese_English_wordvectors.vec")

query = {}
query["Data_type"] = {
    u"$exists": True
}

In [4]:
# For Chinese
cursor = db.Chinese.find(query, no_cursor_timeout=True)
try:
    for doc in cursor:
        each_doc_tokens = []
        for each_sentence in doc["chi_result"]["tokens"]:
            each_doc_tokens += [token for token in each_sentence if token in Concatenated_model.index2word]
        if len(each_doc_tokens) > 0:
            corpus.append({"_id":doc["_id"], "extracted_text":each_doc_tokens, "Class":doc["Class"],
                          "Data_type":doc["Data_type"], "language":"Chinese"})
finally:
    client.close()

In [6]:
%%time
# For English
cursor = db.English.find(query, no_cursor_timeout=True)
try:
    for doc in cursor:
        each_doc_tokens = []
        for each_sentence in doc["nested_token_list"]:
            #each_doc_tokens += [token["lemma_"] for token in each_sentence if token["lemma_"] in Concatenated_model.index2word]
            tokens_from_each_sentence = project_function_for_every_document(each_sentence, want_stop=False,
                                                                        want_alpha=True, want_lemma=True,
                                                                        accept_pos = ["NOUN", "VERB"],
                                                                        use_entity=False)
            tokens_from_each_sentence = [token for token in tokens_from_each_sentence
                                         if token in Concatenated_model.index2word]
            each_doc_tokens += tokens_from_each_sentence
        if len(each_doc_tokens):
            corpus.append({"_id":doc["_id"], "extracted_text":each_doc_tokens, "Class":doc["Class"],
                          "Data_type":doc["Data_type"], "language":"English"})
finally:
    client.close()

CPU times: user 1min 28s, sys: 477 ms, total: 1min 29s
Wall time: 2min 10s


In [7]:
len(corpus)

29994

In [8]:
df = pd.DataFrame(corpus)

In [9]:
print(df.shape)
df.tail(5)

(29994, 5)


,Class,Data_type,_id,extracted_text,language
29989,ECAT,train,5bf42635d3d28003f2bf91e7,"[plan, trade, zone, border, rail, network, lin...",English
29990,GCAT,test,5bf42635d3d28003f2bf91f9,"[number, people, virus, cause, jump, percent, ...",English
29991,CCAT,train,5bf42635d3d28003f2bf91fb,"[agreement, save, airline, bankruptcy, reach, ...",English
29992,ECAT,train,5bf42635d3d28003f2bf9218,"[cash, reserve, carryover, suplus, year, end, ...",English
29993,MCAT,test,5bf42635d3d28003f2bf921b,"[ringgit, fall, surge, dollar, yen, bring, sel...",English


In [10]:
df.to_pickle("/home/ponshane/work_dir/CLTM-Experiments/Data/EN-ZH-MLDoc/tagged_englishAndchinese_corpus_pd.pkl")

# Japanese-English Corpus

In [14]:
#format => [{_id, extracted_text, Class, Data_type, language}, ... ]
corpus = []

# load back model to filter tokens
Concatenated_model = KeyedVectors.load_word2vec_format("/home/ponshane/work_dir/CLTM-Experiments/Data/EN-JP-MLDoc/word-vectors/Japanese_English_wordvectors.vec")

query = {}
query["Data_type"] = {
    u"$exists": True
}

word_set = set(Concatenated_model.index2word)

In [15]:
# For Japanese
cursor = db.Japanese.find(query, no_cursor_timeout=True)
try:
    for doc in cursor:
        each_doc_tokens = []
        # no setence layer in japanese corpus ....
        for each_token in doc["jap_result"]["tokens"]:
            if each_token in word_set:
                each_doc_tokens += [each_token]
        if len(each_doc_tokens) > 0:
            corpus.append({"_id":doc["_id"], "extracted_text":each_doc_tokens, "Class":doc["Class"],
                          "Data_type":doc["Data_type"], "language":"Japanese"})
finally:
    client.close()

In [17]:
%%time
# For English
cursor = db.English.find(query, no_cursor_timeout=True)
try:
    for doc in cursor:
        each_doc_tokens = []
        for each_sentence in doc["nested_token_list"]:
            #each_doc_tokens += [token["lemma_"] for token in each_sentence if token["lemma_"] in Concatenated_model.index2word]
            tokens_from_each_sentence = project_function_for_every_document(each_sentence, want_stop=False,
                                                                        want_alpha=True, want_lemma=True,
                                                                        accept_pos = ["NOUN", "VERB"],
                                                                        use_entity=False)
            tokens_from_each_sentence = [token for token in tokens_from_each_sentence
                                         if token in Concatenated_model.index2word]
            each_doc_tokens += tokens_from_each_sentence
        if len(each_doc_tokens) > 0:
            corpus.append({"_id":doc["_id"], "extracted_text":each_doc_tokens, "Class":doc["Class"],
                          "Data_type":doc["Data_type"], "language":"English"})
finally:
    client.close()

CPU times: user 2min 44s, sys: 500 ms, total: 2min 45s
Wall time: 3min 17s


In [18]:
len(corpus)

29997

In [19]:
df = pd.DataFrame(corpus)

In [21]:
print(df.shape)
df.tail(5)

(29997, 5)


,Class,Data_type,_id,extracted_text,language
29992,ECAT,train,5bf42635d3d28003f2bf91e7,"[plan, trade, zone, border, rail, network, lin...",English
29993,GCAT,test,5bf42635d3d28003f2bf91f9,"[number, people, virus, cause, jump, percent, ...",English
29994,CCAT,train,5bf42635d3d28003f2bf91fb,"[agreement, save, airline, bankruptcy, reach, ...",English
29995,ECAT,train,5bf42635d3d28003f2bf9218,"[cash, reserve, carryover, suplus, year, end, ...",English
29996,MCAT,test,5bf42635d3d28003f2bf921b,"[ringgit, fall, surge, dollar, yen, bring, sel...",English


In [22]:
df.to_pickle("/home/ponshane/work_dir/CLTM-Experiments/Data/EN-JP-MLDoc/tagged_englishAndjapanese_corpus_pd.pkl")